In [1]:
import pandas as pd
import requests
import json
import hashlib
import time

In [2]:
table = pd.read_csv('table.csv')

In [3]:
#script_df = pd.DataFrame(script)
script_df = pd.read_csv('script.csv')

In [4]:
dialogs = {}

In [5]:
dialogs[1] = {'stage':0,'store_features':[]}

In [6]:
payload1 = json.dumps({
         'question': 'What is the price?',
         'context': 'High price, Korean style'
})
res1 = json.loads(requests.post('http://163.14.137.78:1300/getAnswer',data = payload1).text)

In [7]:
res1

{'answer': 'High price', 'end': 10, 'score': 0.5748034119606018, 'start': 0}

In [8]:
def checkType(q,uid):
    sentences = []
    dialogs[uid]['temp_table'] = table
    dialogs[uid]['temp_table']['question'] = q
    for item in dialogs[1]['temp_table']['sentence']:
        sentences.append([q,item])
    payload2 = json.dumps({
         'sentences':sentences
    })
    # infer
    res2 = json.loads(requests.post('http://127.0.0.1:5001/getAnswer',data = payload2).text)
    # process
    dialogs[uid]['result'] = []
    dialogs[uid]['probability'] = []
    for item in eval(res2['content']):
        dialogs[uid]['result'].append(item[0])
        dialogs[uid]['probability'].append(item[1])
    dialogs[uid]['temp_table']['result'] = pd.Series(dialogs[1]['result'])
    dialogs[uid]['temp_table']['probability'] = pd.Series(dialogs[1]['probability'])
    # prepare to return
    try:
        max_fit_prob = dialogs[uid]['temp_table'][dialogs[1]['temp_table'].result == '1']['probability'].max()
        #dialogs[uid]['temp_table'][(dialogs[1]['temp_table'].result == '1') & (dialogs[1]['temp_table'].probability == max_fit_prob)]
        return dialogs[uid]['temp_table'][(dialogs[1]['temp_table'].result == '1') & (dialogs[1]['temp_table'].probability == max_fit_prob)].iloc[0]['meaning']
    except:
        return None

In [9]:
# 語義匹配，需要威宗使用NLP來實現高級功能，return str
def CheckInput(q):
    return 'checked'

In [10]:
# 檢查Context_Type，需要威宗使用NLP來實現高級功能，return str
# def checkType(q):
#     if q == "你好":
#         return "Greeting"
#     if q == "餐廳":
#         return "Shop"
#     if q == "停下":
#         return "Stop"

In [11]:
# 負責最終的推薦
def Rcmd():
    return 'test_shop'

In [29]:
#此方法最終只輸出一個str
def dialog(q,uid):
    # stage == 0
    try:
        # 本地變數
        if dialogs[uid]['stage'] == 0 and checkType(q,1) == 'greeting':
            return '歡迎回到對話！'
        else:
            pass
    except:
        # 本地變數
        dialogs[uid] = {'stage':0,'store_features':[]}
        return '你好，似乎是初次见面，请问需要什么帮助吗？'
    # 檢測打招呼
    # if stage == 0 and CheckType(q) == "Greeting":
    #     session = script_df[script_df.Input == q]
    #     return session.iloc[0]['Output']
    # 詢問店家就這麼做
    if dialogs[uid]['stage'] == 0 and checkType(q,uid) == "shop":
        # 本地變數
        dialogs[uid]['stage'] = 1
        session = script_df[script_df.Input == q]
        return session.iloc[0]['Output']
    # stage == 1
    # 詢問商店特徵就這麼做
    if dialogs[uid]['stage'] == 1 and checkType(q,uid) != 'stop':
        dialogs[uid]['store_features'].append(q)
        return '收到！請問還要補充嗎？'
    elif dialogs[uid]['stage'] == 1 and checkType(q,uid) == 'stop':
        dialogs[uid]['stage'] = 2
        return 'OK，現在幫你推薦！'
    else:
        pass
    # stage == 2
    if dialogs[uid]['stage'] == 2:
        return '不好意思，請問想重新推薦嗎？'

In [30]:
dialogs

{1: {'stage': 0, 'store_features': []}}

In [31]:
print(dialog('你好',1))

歡迎回到對話！


In [32]:
print(dialog('餐廳',1))

你好，請問想要如何了解餐廳：您可以說說價格高低、餐點類型之類的


In [33]:
print(dialog('高價',1))

收到！請問還要補充嗎？


In [34]:
print(dialog('韓式',1))

收到！請問還要補充嗎？


In [35]:
print(dialog('停下',1))

OK，現在幫你推薦！


In [37]:
print(dialog('等一下',1))

不好意思，請問想重新推薦嗎？


In [ ]:
def translate_z2e(query):
    appid = '20220102001044938'
    q = query
    salt='123456'
    key = '4i3lYa4aSyuH2TCcmaBh'
    mid = appid + q + salt + key
    hl = hashlib.md5()
    hl.update(mid.encode(encoding='utf-8'))
    sign = hl.hexdigest()
    url = f'https://api.fanyi.baidu.com/api/trans/vip/translate?q={q}&from=zh&to=en&appid={appid}&salt={salt}&sign={sign}'
    return eval(requests.get(url).text)['trans_result'][0]['dst']

In [ ]:
def translate_e2z(query):
    appid = '20220102001044938'
    q = query
    salt='123456'
    key = '4i3lYa4aSyuH2TCcmaBh'
    mid = appid + q + salt + key
    hl = hashlib.md5()
    hl.update(mid.encode(encoding='utf-8'))
    sign = hl.hexdigest()
    url = f'https://api.fanyi.baidu.com/api/trans/vip/translate?q={q}&from=en&to=zh&appid={appid}&salt={salt}&sign={sign}'
    return eval(requests.get(url).text)['trans_result'][0]['dst']